In [ ]:

distance = mature_biomass_1k.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("distance") \
    .updateMask(age_agbd.select("age")).float().clip(roi)


# Function to buffer and calculate mean
def buffer_and_calculate_mean(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(30), 1))
    mean_value = mature_biomass.reduceRegion(ee.Reducer.mean().unweighted(), buffered.geometry(), 100, \
                                               maxPixels=1e13)
    return buffered.set(mean_value)

pts_dist = distance.reduceRegions(unified_data_fc, ee.Reducer.first().setOutputs(["distance"]))

pts = pts_dist.map(buffer_and_calculate_mean)


